# APIM ❤️ AI Agents

## Multi-agent architecture using Agent-2-Agent connumication protocols (MCP and A2A)
![flow](../../images/mcp-agent-2-agent.gif)
![flow](../../images/a2a-agent-2-agent.gif)

Playground to experiment with [Agent 2 Agent Communication over MCP](https://devblogs.microsoft.com/blog/can-you-build-agent2agent-communication-on-mcp-yes) and [A2A-enabled](https://www.microsoft.com/en-us/microsoft-cloud/blog/2025/05/07/empowering-multi-agent-apps-with-the-open-agent2agent-a2a-protocol/?msockid=3fc737ab34566ad7248a2255359d6b2c) agents with [MCP Tools](https://modelcontextprotocol.io/) with Azure API Management to enable plug & play of tools to LLMs. Leverages the [credential manager](https://learn.microsoft.com/en-us/azure/api-management/credentials-overview) for  managing OAuth 2.0 tokens to backend tools and [client token validation](https://learn.microsoft.com/en-us/azure/api-management/validate-jwt-policy) to ensure end-to-end authentication and authorization.   

This lab includes the following MCP Tools servers:
- Basic oncall service: provides a tool to get a list of random people currently on-call with their status and time zone.
- Basic weather service: provide tools to get cities for a given country and retrieve random weather information for a specified city.

MCP-enabled agents are then deployed within ACA (Azure Container Apps) as both MCP servers and A2A agents, one built with Semantic Kernel, and another with Autogen.

This lab demonstrates the art of the possible of creating hetrogenous multi-agentic system with agents created using multiple orchestrators, and then allowing a single unifying protocol to communicate accross the through APIM for Authn/Authz

### Prerequisites

- [Python 3.13 or later](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [1]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"

apim_sku = 'Basicv2'

openai_resources = [ {"name": "openai1", "location": "uksouth"}]
openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_model_sku = "GlobalStandard"
openai_deployment_name = "gpt-4o-mini"
openai_api_version = "2024-10-21"

build = 0
weather_mcp_server_image = "weather-mcp-server"
weather_mcp_server_src = "../../shared/mcp-servers/weather/http"

oncall_mcp_server_image = "oncall-mcp-server"
oncall_mcp_server_src = "../../shared/mcp-servers/oncall/http"

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 10:09:03.737960 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [2]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 10:09:06.140056 :2s]
👉🏽 Current user: lproux@microsoft.com
👉🏽 Tenant ID: 2b9d9f47-1fb6-400a-a438-39fe7d768649
👉🏽 Subscription ID: d334f2cd-3efd-494e-9fd3-2470b1a13e4c


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [3]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "openAIModelSKU": { "value": openai_model_sku },
        "openAIAPIVersion": { "value": openai_api_version },
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az group show --name lab-mcp-a2a-agents 
👉🏽 Resource group lab-mcp-a2a-agents does not yet exist. Creating the resource group now...
⚙️ Running: az group create --name lab-mcp-a2a-agents --location westeurope --tags source=ai-gateway 
✅ Resource group 'lab-mcp-a2a-agents' created ⌚ 10:09:15.977741 :5s]
⚙️ Running: az deployment group create --name mcp-a2a-agents --resource-group lab-mcp-a2a-agents --template-file main.bicep --parameters params.json 
✅ Deployment 'mcp-a2a-agents' succeeded ⌚ 10:12:03.576062 :47s]


<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [4]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM Gateway URL')
    apim_resource_name = utils.get_deployment_output(output, 'apimResourceName', 'APIM Resource Name')
    apim_subscription_key = utils.get_deployment_output(output, 'apimSubscriptionKey', 'APIM Subscription Key (masked)', False)
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')
    container_registry_name = utils.get_deployment_output(output, 'containerRegistryName', 'Container Registry Name')
    weather_containerapp_resource_name = utils.get_deployment_output(output, 'weatherMCPServerContainerAppResourceName', 'Weather Container App Resource Name')
    oncall_containerapp_resource_name = utils.get_deployment_output(output, 'oncallMCPServerContainerAppResourceName', 'Oncall Container App Resource Name')

    a2a_weather_containerapp_resource_name = utils.get_deployment_output(output, 'a2AWeatherAgentServerContainerAppResourceName', 'A2A (Weather) Agent Container App Resource Name')
    a2a_oncall_containerapp_resource_name = utils.get_deployment_output(output, 'a2AOncallAgentServerContainerAppResourceName', 'A2A (Oncall) Agent Container App Resource Name')

    a2a_weather_a2a_agent_ep = utils.get_deployment_output(output, 'a2AWeatherAgentServerContainerAppFQDN', 'A2A (Weather) Agent Endpoint')
    a2a_oncall_a2a_agent_ep = utils.get_deployment_output(output, 'a2AOncallAgentServerContainerAppFQDN', 'A2A (Oncall) Agent Endpoint')


⚙️ Running: az deployment group show --name mcp-a2a-agents -g lab-mcp-a2a-agents 
✅ Retrieved deployment: mcp-a2a-agents ⌚ 10:12:07.535687 :3s]
👉🏽 APIM Service Id: /subscriptions/d334f2cd-3efd-494e-9fd3-2470b1a13e4c/resourceGroups/lab-mcp-a2a-agents/providers/Microsoft.ApiManagement/service/apim-iauylgmgqsk3i
👉🏽 APIM Gateway URL: https://apim-iauylgmgqsk3i.azure-api.net
👉🏽 APIM Resource Name: apim-iauylgmgqsk3i
👉🏽 APIM Subscription Key (masked): 31bdedaaa0694450b92a34459ea0047b
👉🏽 Application Insights Name: insights-iauylgmgqsk3i
👉🏽 Container Registry Name: acriauylgmgqsk3i
👉🏽 Weather Container App Resource Name: aca-weather-tools-iauylgmgqsk3i
👉🏽 Oncall Container App Resource Name: aca-oncall-tools-iauylgmgqsk3i
👉🏽 A2A (Weather) Agent Container App Resource Name: aca-weather-agent-iauylgmgqsk3i
👉🏽 A2A (Oncall) Agent Container App Resource Name: aca-oncall-agent-iauylgmgqsk3i
👉🏽 A2A (Weather) Agent Endpoint: aca-weather-agent-iauylgmgqsk3i.purplebeach-d45f94a9.westeurope.azurecontainer

<a id='4'></a>
### 4️⃣ Build and deploy the MCP Tools (to be use by both labs)



In [5]:
build = build + 2 # increment the build number

utils.run(f"az acr build --image {weather_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {weather_mcp_server_src}/Dockerfile {weather_mcp_server_src}/. --no-logs", 
          "Weather MCP Server image was successfully built", "Failed to build the Weather MCP Server image")
utils.run(f'az containerapp update -n {weather_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{weather_mcp_server_image}:v0.{build}"', 
          "Weather MCP Server deployment succeeded", "Weather MCP Server deployment failed")

utils.run(f"az acr build --image {oncall_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {oncall_mcp_server_src}/Dockerfile {oncall_mcp_server_src}/. --no-logs", 
          "Oncall MCP Server image was successfully built", "Failed to build the Oncall MCP Server image")
utils.run(f'az containerapp update -n {oncall_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{oncall_mcp_server_image}:v0.{build}"', 
          "Oncall MCP Server deployment succeeded", "Oncall MCP Server deployment failed")


⚙️ Running: az acr build --image weather-mcp-server:v0.2 --resource-group lab-mcp-a2a-agents --registry acriauylgmgqsk3i --file ../../shared/mcp-servers/weather/http/Dockerfile ../../shared/mcp-servers/weather/http/. --no-logs 
✅ Weather MCP Server image was successfully built ⌚ 10:13:15.676921 :8s]
⚙️ Running: az containerapp update -n aca-weather-tools-iauylgmgqsk3i -g lab-mcp-a2a-agents --image "acriauylgmgqsk3i.azurecr.io/weather-mcp-server:v0.2" 
✅ Weather MCP Server deployment succeeded ⌚ 10:13:36.502738 :20s]
⚙️ Running: az acr build --image oncall-mcp-server:v0.2 --resource-group lab-mcp-a2a-agents --registry acriauylgmgqsk3i --file ../../shared/mcp-servers/oncall/http/Dockerfile ../../shared/mcp-servers/oncall/http/. --no-logs 
✅ Oncall MCP Server image was successfully built ⌚ 10:14:45.708539 :9s]
⚙️ Running: az containerapp update -n aca-oncall-tools-iauylgmgqsk3i -g lab-mcp-a2a-agents --image "acriauylgmgqsk3i.azurecr.io/oncall-mcp-server:v0.2" 
✅ Oncall MCP Server deployme

<a id='testconnection'></a>
### 🧪 Test the connection to the MCP Tools' servers and List Tools

💡 To integrate MCP servers in VS Code, use the MCP server URL  `../mcp ` for configuration in GitHub Copilot Agent Mode

At this stage we're just testing that our MCP tools servers work:

![flow](../../images/agent-2-mcp.gif)



In [6]:
import os, json, asyncio, time, requests
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
import nest_asyncio
nest_asyncio.apply()

async def list_tools(server_url, authorization_header = None):
    headers = {"Authorization": authorization_header} if authorization_header else None
    streams = None
    session = None
    tools = []
    try:
        streams_ctx = streamablehttp_client(server_url, headers)
        streams = await streams_ctx.__aenter__()
        session_ctx = ClientSession(streams[0], streams[1])
        session = await session_ctx.__aenter__()
        await session.initialize()
        response = await session.list_tools()
        tools = response.tools
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        # Ensure session and streams are closed if they were opened
        if session is not None:
            await session_ctx.__aexit__(None, None, None) # type: ignore
        if streams is not None:
            await streams_ctx.__aexit__(None, None, None) # type: ignore
    if tools:
        print(f"✅ Connected to server {server_url}")
        print("⚙️ Tools:")
        for tool in tools:
            print(f"  - {tool.name}")
            print(f"     Input Schema: {tool.inputSchema}")

try:
    asyncio.run(list_tools(f"{apim_resource_gateway_url}/weather"))
    asyncio.run(list_tools(f"{apim_resource_gateway_url}/oncall"))
finally:
    print(f"✅ Connection closed")


✅ Connected to server https://apim-iauylgmgqsk3i.azure-api.net/weather
⚙️ Tools:
  - get_cities
     Input Schema: {'properties': {'country': {'type': 'string'}}, 'required': ['country'], 'type': 'object'}
  - get_weather
     Input Schema: {'properties': {'city': {'type': 'string'}}, 'required': ['city'], 'type': 'object'}
✅ Connection closed


CancelledError: Cancelled via cancel scope 24ef0bd2150 by <Task cancelling name='Task-20' coro=<list_tools() running at C:\Users\lproux\AppData\Local\Temp\ipykernel_20136\902043953.py:25>>

#### One Agent + 2 MCP Tools

In [ ]:
import asyncio
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

inference_api_path = ""
inference_api_version = "2025-03-01-preview"

user_input = "What's the current weather in Lisbon and London? and who's oncall today?"

weather_plugin = MCPStreamableHttpPlugin(
    name="Weather",
    url=f"{apim_resource_gateway_url}/weather",
    description="Weather Plugin",
    )

oncall_plugin = MCPStreamableHttpPlugin(
    name="OnCall",
    url=f"{apim_resource_gateway_url}/oncall",
    description="OnCall Plugin",
)

await weather_plugin.connect()  # Ensure the plugin is connected before using it
await oncall_plugin.connect()  # Ensure the plugin is connected before using it

agent = ChatCompletionAgent(
    service=AzureChatCompletion(
        endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
        api_key=apim_subscription_key,
        api_version=inference_api_version,                
        deployment_name=openai_model_name  # Use the first model from the models_config
    ),
    name="IssueAgent",
    instructions="Answer questions using the tools at your disposal.",
    plugins=[weather_plugin, oncall_plugin],
)

thread: ChatHistoryAgentThread | None = None

print(f"# User: {user_input}")
# 2. Invoke the agent for a response
response = await agent.get_response(messages=user_input, thread=thread)
print(f"# {response.name}: {response} ")
thread = response.thread # type: ignore

# 3. Cleanup: Clear the thread
await thread.delete() if thread else None



<a id='inspector'></a>
### 🧪 (optional) Use the [MCP Inspector](https://modelcontextprotocol.io/docs/tools/inspector) for testing and debugging the MCP servers

#### Execute the following steps:
1. Execute `npx @modelcontextprotocol/inspector` in a terminal
2. Open the provided URL in a browser
3. Set the transport type as SSE
4. Provide the MCP server url and click connect
5. Select the "Tools" tab to see and run the available tools

<a id='agent-to-agent'></a>
### 5️⃣ Experiment with agent-to-agent communication

This is what we're all here for!! Choose a path :) 

- Use notebook to exexoeriment with agent-to-agent communication [(using MCP Protocol)](mcp-agent-as-mcp-server.ipynb)
- Use notebooks experiment with a2a-enabled agents [(using a2a Protocol)](mcp-agent-as-a2a-server.ipynb)

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.